In [1]:
!pip uninstall -y numpy torch torchaudio torchvision lightning pytorch-lightning nvidia-cudnn-cu12 -q

In [ ]:
!pip install numpy==1.26.4

In [ ]:
!pip install torch torchaudio torchvision  # 코랩 최신 버전 사용


In [ ]:
!pip install lightning pytorch-lightning  # 코랩 최신 버전 사용


In [ ]:
!pip install faster-whisper

In [6]:
!pip install torchcodec  # 추가!


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.7 MB/s eta 0:00:00


In [ ]:
!pip install nemo-toolkit[asr]

In [1]:
import os
import json
import torch
import torchaudio
from pathlib import Path
from faster_whisper import WhisperModel
from nemo.collections.asr.models.msdd_models import NeuralDiarizer
from omegaconf import OmegaConf

[NeMo W 2025-11-06 03:08:43 nemo_logging:405] Megatron num_microbatches_calculator not found, using Apex version.
[NeMo W 2025-11-06 03:08:46 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2025-11-06 03:08:46 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2025-11-06 03:08:46 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2025-11-06 03:08:46 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(dbl)p?( \(default\))?$', token):
    


In [2]:
from google.colab import files

# 파일 업로드
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print(f"업로드된 파일: {audio_path}")

Saving 1105_오전회의.m4a to 1105_오전회의.m4a
업로드된 파일: 1105_오전회의.m4a


# 오디오 변환 (M4A → WAV 16kHz mono)

In [7]:
# 출력 디렉토리 생성
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# WAV로 변환
wav_path = os.path.join(output_dir, "audio.wav")

print("🔄 오디오 변환 중...")

# ffmpeg로 직접 변환 (torchaudio 버전 문제 우회)
import subprocess
subprocess.run([
    'ffmpeg', '-i', audio_path,
    '-ar', '16000',  # 16kHz
    '-ac', '1',      # mono
    '-y',            # overwrite
    wav_path
], capture_output=True)

# 변환된 파일 로드
waveform, sample_rate = torchaudio.load(wav_path)

duration = waveform.shape[1] / sample_rate
print(f"✅ 변환 완료: {wav_path}")
print(f"   샘플레이트: {sample_rate}Hz")
print(f"   길이: {duration:.1f}초")

🔄 오디오 변환 중...
✅ 변환 완료: output/audio.wav
   샘플레이트: 16000Hz
   길이: 933.5초


# Whisper로 음성 전사

In [8]:
# Whisper 모델 로드 (large-v2 추천, 빠른 테스트는 medium 사용)
model_size = "large-v3"  # tiny, base, small, medium, large-v2, large-v3

print(f"Whisper {model_size} 모델 로딩...")
whisper_model = WhisperModel(model_size, device="cuda", compute_type="float16")

# 전사 실행
print("음성 전사 중...")
segments, info = whisper_model.transcribe(
    wav_path,
    language="ko",
    beam_size=5,
    word_timestamps=True
)

# 결과를 리스트로 저장
transcription = []
for segment in segments:
    transcription.append({
        'start': segment.start,
        'end': segment.end,
        'text': segment.text,
        'words': [
            {'word': w.word, 'start': w.start, 'end': w.end}
            for w in segment.words
        ] if segment.words else []
    })

print(f"전사 완료: {len(transcription)} 세그먼트")
print(f"언어: {info.language}")

# 미리보기
print("\n[전사 미리보기]")
for i, seg in enumerate(transcription[:3]):
    print(f"{i+1}. [{seg['start']:.2f}s - {seg['end']:.2f}s] {seg['text']}")

Whisper large-v3 모델 로딩...


config.json: 0.00B [00:00, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

음성 전사 중...
전사 완료: 377 세그먼트
언어: ko

[전사 미리보기]
1. [0.00s - 1.90s]  시작되면 말하기 시작
2. [1.90s - 2.56s]  시작
3. [2.56s - 3.80s]  네, 눌렀습니다


# NeMo로 화자 분리

In [21]:
import wget

# NeMo용 manifest 파일 생성
manifest_path = os.path.join(output_dir, "input_manifest.json")
with open(manifest_path, 'w') as f:
    json.dump({
        "audio_filepath": wav_path,
        "offset": 0,
        "duration": None,
        "label": "infer",
        "text": "-",
        "rttm_filepath": None,  # 추가!
        "uem_filepath": None    # 추가!
    }, f)
    f.write('\n')

# NeMo 공식 config 다운로드
CONFIG_URL = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_telephonic.yaml"
CONFIG_PATH = os.path.join(output_dir, "diar_infer_telephonic.yaml")

if not os.path.exists(CONFIG_PATH):
    print("Config 다운로드 중...")
    CONFIG_PATH = wget.download(CONFIG_URL, output_dir)

# Config 로드
config = OmegaConf.load(CONFIG_PATH)

# 필수 항목만 수정
config.num_workers = 0
config.diarizer.manifest_filepath = manifest_path
config.diarizer.out_dir = output_dir

print("✅ NeMo 설정 완료")
print(f"사용 모델: {config.diarizer.msdd_model.model_path}")

✅ NeMo 설정 완료
사용 모델: diar_msdd_telephonic


In [15]:
import os
os.environ['LD_LIBRARY_PATH'] = ''
import torch
torch.backends.cudnn.enabled = False  # cuDNN 비활성화


In [22]:
# 화자 분리 실행
print("화자 분리 중...")
msdd_model = NeuralDiarizer(cfg=config)
msdd_model.diarize()

# RTTM 파일 읽기
rttm_path = os.path.join(output_dir, "pred_rttms", "audio.rttm")

speaker_segments = []
with open(rttm_path, 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 8:
            start = float(parts[3])
            duration = float(parts[4])
            speaker = parts[7]
            speaker_segments.append({
                'start': start,
                'end': start + duration,
                'speaker': speaker
            })

del msdd_model
torch.cuda.empty_cache()

print(f"화자 분리 완료: {len(speaker_segments)} 세그먼트")

# 화자 수 확인
unique_speakers = set([s['speaker'] for s in speaker_segments])
print(f"감지된 화자 수: {len(unique_speakers)}")
print(f"화자 레이블: {sorted(unique_speakers)}")

화자 분리 중...
[NeMo I 2025-11-06 03:22:48 nemo_logging:393] Loading pretrained diar_msdd_telephonic model from NGC
[NeMo I 2025-11-06 03:22:48 nemo_logging:393] Found existing object /root/.cache/torch/NeMo/NeMo_2.5.2/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2025-11-06 03:22:48 nemo_logging:393] Re-using file from: /root/.cache/torch/NeMo/NeMo_2.5.2/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo
[NeMo I 2025-11-06 03:22:48 nemo_logging:393] Instantiating model from pre-trained checkpoint


[NeMo W 2025-11-06 03:22:50 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: true
    
[NeMo W 2025-11-06 03:22:50 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: false
    
[NeMo W 2025-11-06 03:22:50 nemo_logging:405] Please call the ModelPT.setup_test_data() or ModelPT

[NeMo I 2025-11-06 03:22:50 nemo_logging:393] PADDING: 16
[NeMo I 2025-11-06 03:22:50 nemo_logging:393] PADDING: 16
[NeMo I 2025-11-06 03:22:51 nemo_logging:393] Model EncDecDiarLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_2.5.2/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2025-11-06 03:22:51 nemo_logging:393] PADDING: 16
[NeMo I 2025-11-06 03:22:52 nemo_logging:393] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2025-11-06 03:22:52 nemo_logging:393] Found existing object /root/.cache/torch/NeMo/NeMo_2.5.2/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2025-11-06 03:22:52 nemo_logging:393] Re-using file from: /root/.cache/torch/NeMo/NeMo_2.5.2/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2025-11-06 03:22:52 nemo_logging:393] Instantiating model from pre-trained checkpoint


[NeMo W 2025-11-06 03:22:52 nemo_logging:405] Please use the EncDecSpeakerLabelModel instead of this model. EncDecClassificationModel model is kept for backward compatibility with older models.
[NeMo W 2025-11-06 03:22:52 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_

[NeMo I 2025-11-06 03:22:52 nemo_logging:393] PADDING: 16
[NeMo I 2025-11-06 03:22:52 nemo_logging:393] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_2.5.2/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2025-11-06 03:22:52 nemo_logging:393] Multiscale Weights: [1, 1, 1, 1, 1]
[NeMo I 2025-11-06 03:22:52 nemo_logging:393] Clustering Parameters: {
        "oracle_num_speakers": false,
        "max_num_speakers": 8,
        "enhanced_count_thres": 80,
        "max_rp_threshold": 0.25,
        "sparse_search_volume": 30,
        "maj_vote_spk_count": false,
        "chunk_cluster_count": 50,
        "embeddings_per_chunk": 10000
    }


...

[NeMo I 2025-11-06 03:23:41 nemo_logging:393] Number of files to diarize: 1


[NeMo W 2025-11-06 03:23:41 nemo_logging:405] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2025-11-06 03:23:41 nemo_logging:393] Number of files to diarize: 1


[NeMo W 2025-11-06 03:23:41 nemo_logging:405] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2025-11-06 03:23:41 nemo_logging:393]   
    
화자 분리 완료: 476 세그먼트
감지된 화자 수: 4
화자 레이블: ['speaker_0', 'speaker_1', 'speaker_2', 'speaker_3']


# 화자와 텍스트 결합

In [23]:
# 단어별로 화자 할당
print("화자 할당 중...")

diarized = []
for segment in transcription:
    segment_result = {
        'start': segment['start'],
        'end': segment['end'],
        'text': segment['text'],
        'words': []
    }

    for word in segment.get('words', []):
        word_start = word['start']
        word_end = word['end']

        # 해당 단어 시간에 맞는 화자 찾기
        speaker = None
        max_overlap = 0

        for spk_seg in speaker_segments:
            overlap_start = max(word_start, spk_seg['start'])
            overlap_end = min(word_end, spk_seg['end'])
            overlap = max(0, overlap_end - overlap_start)

            if overlap > max_overlap:
                max_overlap = overlap
                speaker = spk_seg['speaker']

        segment_result['words'].append({
            'word': word['word'],
            'start': word_start,
            'end': word_end,
            'speaker': speaker
        })

    diarized.append(segment_result)

print("화자 할당 완료")

화자 할당 중...
화자 할당 완료


In [25]:
# 화자별로 텍스트 그룹화
output_lines = []
current_speaker = None
current_text = []

for segment in diarized:
    for word in segment['words']:
        if word['speaker'] != current_speaker:
            # 화자가 바뀌면 이전 내용 저장
            if current_text:
                output_lines.append(
                    f"{current_speaker}: {' '.join(current_text)}"
                )
                current_text = []
            current_speaker = word['speaker']

        current_text.append(word['word'].strip())

# 마지막 남은 내용 저장
if current_text:
    output_lines.append(
        f"{current_speaker}: {' '.join(current_text)}"
    )

formatted_text = '\n\n'.join(output_lines)

# 파일로 저장
output_path = os.path.join(output_dir, "transcript.txt")
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(formatted_text)

print(f"결과 저장 완료: {output_path}")

결과 저장 완료: output/transcript.txt


In [26]:
print("=" * 50)
print("화자분리 결과")
print("=" * 50)
print(formatted_text[:1000])  # 첫 1000자만 출력
print("\n...")
print(f"\n전체 내용은 {output_path} 파일을 확인하세요.")

화자분리 결과
speaker_0: 시작되면 말하기 시작

speaker_1: 시작

speaker_0: 네, 눌렀습니다 네 다시 제 것부터 간략히 말씀드리면 저는 세 개의 주제에 대해서 찾아보고 있었는데요 어제는 주로 저는 작가 문체로 자동 교정해주는 것을 염두에 두고 시작해봤는데 결과적으로 다 마음에 들지만 데이터도 쉽게 쉽게 구한다는 말은 쉽게 꿀 빠하려는 의미보다는 쉽게 더 강력한 데이터를 구해서 우리 모델한테 더 좋은 먹이를 주고 싶다는 연료를 주고 싶다는 마음이 컸어요 그래가지고 그러다 보니까 또 실시간 세계 경제 뉴스나 바로 하자 자동 태깅 쪽으로 갈 수밖에 좀 없는 판단을 했고요 자발적으로 그래서 이제 또 그러면 이제 그 두 개 중에 좀 더 제가 힘을 싣고 싶었던 거는 좀 더 이렇게 어필링한 어떤 포트폴리오를 위해서 바로 하자 자동 태깅을 좀 더 해보자 그리고 팀원들의 어떤 의견 수렴도 전반적으로 그런 기회를 이루었던 것 같아서 같이 그냥 동참해서 가는 게 좋을 것 같아서 신속한 의사 결정을 위해서 그래서 이제 자료를 조사해봤더니 국립국어원에서 상당히 강력하고 풍부한 데이터를 제공해 주는 것 같습니다 근데 이게 지금 음성 형태인지 아니면 그냥 포퍼스라니까 그냥 어떤 그 텍스트 형태인지 좀 더 뜯어보고 공유 드릴 예정이고요 그리고 이제 두 번째로 제가 지금 염두에 두고 있는 게 AI 허브의 이제 어떤 대화 데이터는 정말 무궁무진하고 많더라고요 그리고 상황도 다양해요 예를 들면 긴급 출동 상황 뭐 공항 주변 소음 상황 뭐 공사장 상황 그리고 그런 어떤 노이즈가 포함됐냐 안 됐냐를 떠나서도 뭐

speaker_1: 엄청

speaker_0: 많더라고요 뭐 아동의 대화 이런 거 그래서 우리가 우리에게 필요한 데이터로 보여진 게 많아서 좀 이제 많이 스크랩 해놨으니까 이따 또 한번 리스트업해서 공유하도록 하고 어쨌든 그래서 결과적으로 저는 바라자 자동 태깅 쪽으로 데이터를 좀 많이 일단 구비해놨다 정도가 볼 수 있고 여기서 어 이런 데이터는 충분하네요 라고

In [ ]:
from google.colab import files

# 결과 파일 다운로드
files.download(output_path)